In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-17 02:13:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-01-17 02:13:22 (11.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [5]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2R07E5PNXEUO5|          4|            0|          0|   N|                Y|
|R27BA52AKWMWN3|          5|            0|          0|   N|                Y|
| RLF8DOID2KD5O|          3|            0|          0|   N|     

In [7]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [9]:
total_votes_df= vine_df.filter(col('total_votes')>= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [13]:
clean_total_votes_df= total_votes_df.dropna()

In [17]:
votes_df= clean_total_votes_df.withColumn("% of Total Votes", col("helpful_votes")/col("total_votes")).alias("% of Total Votes").filter(col("% of Total Votes")>=0.5)
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  % of Total Votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|0.9183673469387755|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|               1.0|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|0.5925925925925926|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|0.9047619047619048|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|0.8636363636363636|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|               1.0|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|0.9655172413793104|
| R6OD85TMEMHQQ|          5|  

In [18]:
vine_program_paid_df= votes_df.filter(col("vine")=="Y")
vine_program_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  % of Total Votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|0.9545454545454546|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|0.8947368421052632|
|R2MPEQ4SPTEQNS|          4|          180|        184|   Y|                N|0.9782608695652174|
| RIR0D3KJ0CQ31|          4|           21|         21|   Y|                N|               1.0|
|R1SPWJDHUWWC5E|          5|           88|         98|   Y|                N|0.8979591836734694|
|R1X6M5XA3FT98W|          5|           24|         26|   Y|                N|0.9230769230769231|
|R37VCW6HA0Z72T|          5|           27|         28|   Y|                N|0.9642857142857143|
|R2XRYNV2SY3ZKL|          5|  

In [19]:
vine_program_unpaid_df= votes_df.filter(col("vine")=="N")
vine_program_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  % of Total Votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|0.9183673469387755|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|               1.0|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|0.5925925925925926|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|0.9047619047619048|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|0.8636363636363636|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|               1.0|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|0.9655172413793104|
| R6OD85TMEMHQQ|          5|  

In [29]:
from pyspark.sql.functions import count, when
summary_vine_votes_df= votes_df.groupby("vine").agg(count(col("vine")).alias("total_reviews"),
count(when(col("star_rating")==5, True)).alias("total_5_star_ratings"),
(count(when(col("star_rating") == 5, True))/count(col("vine"))).alias("%_5_star_reviews")).show()

+----+-------------+--------------------+------------------+
|vine|total_reviews|total_5_star_ratings|  %_5_star_reviews|
+----+-------------+--------------------+------------------+
|   Y|           22|                  13|0.5909090909090909|
|   N|        26987|               14475|0.5363693630266425|
+----+-------------+--------------------+------------------+

